### Libraries ##

In [53]:
import os
os.listdir('.')


import pandas as pd
import numpy as np
import random

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import transformers
from transformers import (BertTokenizer,AutoTokenizer, BertForSequenceClassification, 
                          AdamW, get_linear_schedule_with_warmup)
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

## dataset ##

In [55]:
df = pd.read_csv('smile-annotations-final.csv', names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

display('head', df.head())

display('category counts', df.category.value_counts())

df = df[~df.category.str.contains('\|')]

df = df[df.category != 'nocode']

display('category counts after cleanup', df.category.value_counts())

'head'

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


'category counts'

category
nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: count, dtype: int64

'category counts after cleanup'

category
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: count, dtype: int64

In [56]:
### creating labels ###

possible_labels = df.category.unique()

## creating a dict to convert string categories to numbers ##

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [57]:
### splitting data into training and validation sets ###

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

#### Tokenizing data

In [23]:
# Using the BERT tokenizer from the 'bert-base-uncased' model
# and setting do_lower_case to True to ensure all text is lowercased
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

# Encoding the text data in the training set using batch_encode_plus
# This method tokenizes and encodes a batch of sequences, adding special tokens,
# padding the sequences to the same length, and returning PyTorch tensors
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,            # Extracting text data for training
    add_special_tokens=True,                          # Adding special tokens like [CLS] and [SEP]
    return_attention_mask=True,                      # Returning attention masks to focus on actual tokens
    pad_to_max_length=True,                          # Padding sequences to the same length
    max_length=256,                                   # Maximum length of each sequence
    return_tensors='pt'                               # Returning PyTorch tensors
)

# Encoding the text data in the validation set using batch_encode_plus
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,              # Extracting text data for validation
    add_special_tokens=True,                          # Adding special tokens like [CLS] and [SEP]
    return_attention_mask=True,                      # Returning attention masks to focus on actual tokens
    pad_to_max_length=True,                          # Padding sequences to the same length
    max_length=256,                                   # Maximum length of each sequence
    return_tensors='pt'                               # Returning PyTorch tensors
)

# Extracting input IDs, attention masks, and labels for the training set
input_ids_train = encoded_data_train['input_ids']     # Input IDs representing tokenized text
attention_masks_train = encoded_data_train['attention_mask']  # Attention masks indicating which tokens to attend to
labels_train = torch.tensor(df[df.data_type=='train'].label.values)  # Labels for the training set

# Extracting input IDs, attention masks, and labels for the validation set
input_ids_val = encoded_data_val['input_ids']         # Input IDs representing tokenized text
attention_masks_val = encoded_data_val['attention_mask']   # Attention masks indicating which tokens to attend to
labels_val = torch.tensor(df[df.data_type=='val'].label.values)   # Labels for the validation set

# Creating PyTorch datasets for training and validation
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)  # Training dataset
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)          # Validation dataset

### Setting up BERT Pretrained Model

In [33]:
# Initializing the BERT model for sequence classification from the pre-trained 'bert-base-uncased' model
# Specifying the number of labels in the output layer based on the length of the label dictionary
# Setting output_attentions and output_hidden_states to False to exclude additional outputs
# Setting resume_download to True to resume download if interrupted
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False,
                                                      resume_download=True)

# Defining the batch size for training and validation
batch_size = 32

# Creating data loaders for training and validation sets
# Using RandomSampler for training data and SequentialSampler for validation data
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Initializing the AdamW optimizer with the BERT model parameters
# Setting the learning rate to 2e-5 and epsilon to 1e-8
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)

# Defining the number of epochs for training
epochs = 7

# Creating a linear scheduler with warmup for adjusting learning rates during training
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Defining a function to calculate the F1 score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Defining a function to calculate accuracy per class
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [44]:
### assigning seed to be able to reproduce results ###
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [45]:
# Checking for GPU availability and assigning the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Moving the model to the selected device
print(device)  # Printing the device (GPU or CPU) being used

# Defining the evaluation function for the validation set
def evaluate(dataloader_val):

    model.eval()  # Setting the model to evaluation mode
    
    loss_val_total = 0  # Initializing total validation loss
    predictions, true_vals = [], []  # Lists to store predictions and true values
    
    # Iterating through batches in the validation dataloader
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)  # Moving batch tensors to the device
        
        inputs = {'input_ids':      batch[0],      # Input token IDs
                  'attention_mask': batch[1],      # Attention masks
                  'labels':         batch[2],      # Labels
                 }

        with torch.no_grad():  # Disabling gradient calculation        
            outputs = model(**inputs)  # Forward pass
            
        loss = outputs[0]  # Extracting loss value from the output
        logits = outputs[1]  # Predicted logits
        loss_val_total += loss.item()  # Accumulating validation loss

        logits = logits.detach().cpu().numpy()  # Detaching logits from computation graph and moving to CPU
        label_ids = inputs['labels'].cpu().numpy()  # Moving label IDs to CPU
        predictions.append(logits)  # Appending predictions to the list
        true_vals.append(label_ids)  # Appending true values to the list
    
    loss_val_avg = loss_val_total/len(dataloader_val)  # Calculating average validation loss
    
    # Concatenating predictions and true values to form arrays
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals  # Returning validation loss, predictions, and true values

# Training loop for each epoch
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()  # Setting the model to training mode
    
    loss_train_total = 0  # Initializing total training loss

    # Progress bar for training epoch
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()  # Resetting gradients
        
        batch = tuple(b.to(device) for b in batch)  # Moving batch tensors to the device
        
        inputs = {'input_ids':      batch[0],      # Input token IDs
                  'attention_mask': batch[1],      # Attention masks
                  'labels':         batch[2],      # Labels
                 }       

        outputs = model(**inputs)  # Forward pass
        
        loss = outputs[0]  # Extracting loss value from the output
        loss_train_total += loss.item()  # Accumulating training loss
        loss.backward()  # Backpropagation

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clipping gradients to prevent explosion
        
        optimizer.step()  # Optimizer step
        scheduler.step()  # Scheduler step
        
        # Updating progress bar with current training loss
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')  # Saving model after each epoch
        
    tqdm.write(f'\nEpoch {epoch}')  # Printing current epoch
    
    loss_train_avg = loss_train_total/len(dataloader_train)  # Calculating average training loss
    tqdm.write(f'Training loss: {loss_train_avg}')  # Printing training loss
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)  # Evaluating on validation set
    val_f1 = f1_score_func(predictions, true_vals)  # Calculating F1 score
    tqdm.write(f'Validation loss: {val_loss}')  # Printing validation loss
    tqdm.write(f'F1 Score (Weighted): {val_f1}')  # Printing F1 score


cpu


### Training

In [47]:
# Training loop for each epoch
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()  # Setting the model to training mode
    
    loss_train_total = 0  # Initializing total training loss

    # Progress bar for training epoch
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()  # Resetting gradients
        
        batch = tuple(b.to(device) for b in batch)  # Moving batch tensors to the device
        
        inputs = {'input_ids':      batch[0],      # Input token IDs
                  'attention_mask': batch[1],      # Attention masks
                  'labels':         batch[2],      # Labels
                 }       

        outputs = model(**inputs)  # Forward pass
        
        loss = outputs[0]  # Extracting loss value from the output
        loss_train_total += loss.item()  # Accumulating training loss
        loss.backward()  # Backpropagation

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clipping gradients to prevent explosion
        
        optimizer.step()  # Optimizer step
        scheduler.step()  # Scheduler step
        
        # Updating progress bar with current training loss
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')  # Saving model after each epoch
        
    tqdm.write(f'\nEpoch {epoch}')  # Printing current epoch
    
    loss_train_avg = loss_train_total/len(dataloader_train)  # Calculating average training loss
    tqdm.write(f'Training loss: {loss_train_avg}')  # Printing training loss
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)  # Evaluating on validation set
    val_f1 = f1_score_func(predictions, true_vals)  # Calculating F1 score
    tqdm.write(f'Validation loss: {val_loss}')  # Printing validation loss
    tqdm.write(f'F1 Score (Weighted): {val_f1}')  # Printing F1 score


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0078382968902588
Validation loss: 0.7884764501026699
F1 Score (Weighted): 0.6656119824269878


Epoch 2:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.6746107473969459
Validation loss: 0.5846196285315922
F1 Score (Weighted): 0.765966003944399


...


Epoch 5
Training loss: 0.26260226033627987
Validation loss: 0.5020982708249774
F1 Score (Weighted): 0.8417100660606903


Epoch 6:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.2216916624456644
Validation loss: 0.5098877165998731
F1 Score (Weighted): 0.8359704467578306


Epoch 7:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 7
Training loss: 0.1957705019041896
Validation loss: 0.5016267831836428
F1 Score (Weighted): 0.8326666441889475


### Validation set results

In [49]:
# Loading the trained model state dictionary from the file 'finetuned_BERT_epoch_7.model'
# Loading onto the CPU since inference is being performed
model.load_state_dict(torch.load('finetuned_BERT_epoch_7.model', map_location=torch.device('cpu')))

# Evaluating the validation set using the trained model
# Ignoring the returned loss as it's not needed for this evaluation
# Extracting predictions and true values
_, predictions, true_vals = evaluate(dataloader_validation)

# Calculating and printing accuracy per class based on predictions and true values
accuracy_per_class(predictions, true_vals)

<All keys matched successfully>